In [108]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))
import networkx as nx
import random
import jReversion as jR
import robustness as rb
import pandas as pd
import numpy as np
import par_helper as ph

In [69]:
def gen_random_boolean_network_er_kauffman(n=30, p=0.3, bias=0.5):
    '''

    The code is originated from the read_ndetwork() by Colin Campbell.
    '''
    def clean_states(x):
        #cleans binary representation of node input states
        out=x[2:]                                                               # Strip leading 0b
        return '0'*(len(inf)-len(out))+out                                      # Append leading 0's as needed

    while True:
        g = nx.generators.random_graphs.gnp_random_graph(n=n, p=p, directed=True)
        
        input_node = [node for node in g.nodes if g.in_degree(node) == 0]
        output_node = [node for node in g.nodes if g.out_degree(node) == 0]
        if len(input_node) == 0:              # at least one input node
            random_node = random.randrange(n)
            g.remove_edges_from(list(g.in_edges(random_node)))
            input_node = [node for node in g.nodes if g.in_degree(node) == 0]
            
        if len(output_node) == 0:            # at least one output node
            random_node = random.randrange(n)
            g.remove_edges_from(list(g.out_edges(random_node)))
            output_node = [node for node in g.nodes if g.out_degree(node) == 0]
        
        if len(g.subgraph(input_node + output_node).edges) > 0: continue # output nodes are connected to input nodes
                    
        if nx.is_weakly_connected(g): break

    for n in g.nodes:
        inf = list(g.predecessors(n))
        if len(inf) > 0: 
            g.nodes[n]['update_nodes'] = inf.copy()
            g.nodes[n]['update_rules'] = {}

            bool_states = map(bin,range(2**len(inf)))
            bool_states = map(clean_states,bool_states)
            for j in bool_states:
                g.nodes[n]['update_rules'][j] = int(random.random() < bias)    # Store outcome for every possible input
        else:
#             g.add_edge(n, n)
            g.nodes[n]['update_nodes'] = [n]
            g.nodes[n]['update_rules'] = {'0': 0, '1': 1}

    return g,list(g.nodes)

In [96]:
test, _ = gen_random_boolean_network_er_kauffman()
print(len(test.edges)/len(test))

8.166666666666666


In [125]:
results = pd.DataFrame(columns=['network_idx', 'size_of_network', 'num_link', 'C0', 'C1', 'C2', 'C3', 'robustness_pa', 'robustness_ip', 'redundancy', 'deterministic_io','nondeterministic_io'])

In [138]:
num_of_networks = 100
size_of_network = 25
link_probability = 0.1
sampling_num = 10
num_of_worker = 5
prefix, suffix = 'n', 'n'
for network_idx in range(num_of_networks):
    g, read_nodes = gen_random_boolean_network_er_kauffman(n=size_of_network, p=link_probability)
    mapping = {}  # nodename to number index
    inverse_mapping = {}  # number index to nodename
    read_nodes_dict = {}
    inverse_read_nodes_dict = {}
    for i, node in enumerate(read_nodes):
        index = prefix + str(i) + suffix
        mapping[str(node)] = index
        inverse_mapping[index] = str(node)
        mapping['~' + str(node)] = '~' + index
        inverse_mapping['~' + index] = '~' + str(node)
        read_nodes_dict[i] = str(node)
        inverse_read_nodes_dict[str(node)] = i
    
    input_nodes = [node for node in g.nodes if g.in_degree(node) == 0]
    output_nodes = [node for node in g.nodes if g.out_degree(node) == 0]
    
    num_inputs = len(input_nodes)
    num_input_conditions = 2 ** num_inputs
    input_conditions = np.ndarray((num_inputs, 2), dtype=object)
    for idx, input_node in enumerate(input_nodes):
        input_conditions[idx, 0] = '~' + str(input_node)
        input_conditions[idx, 1] = str(input_node)
    
    
    output_nodes_ex = []
    output_nodes_ex.extend([read_nodes_dict[idx] for idx in output_nodes])
    output_nodes_ex.extend(['~'+read_nodes_dict[idx] for idx in output_nodes])
    input_nodes_ex = input_conditions.reshape(num_inputs * 2, ).tolist()
    
    GExpanded = ph.par_get_expanded_network(g, prefix='n', suffix='n', worker=num_of_worker)
    TempGIOW = jR.get_input_output_relation(GExpanded, mapping, inverse_mapping, input_conditions, output_nodes_ex,
                                            constant_nodes=[])
    LDOIs = TempGIOW['LDOIs']
    GeneLDOIs = TempGIOW['gene_LDOIs']
    Conflicts = TempGIOW['conflicts']
    GeneConflicts = TempGIOW['gene_conflicts']
    IORelation = TempGIOW['io_relation']
    GRemained = TempGIOW['G_remained']

    R0 = jR.identifying_r0_mutations_ldoi(GExpanded, output_nodes_ex, mapping, inverse_mapping, input_conditions, IORelation)
    # DM = jR.identifying_disconnecting_mutations(GExpanded, InputNodes, OutputNodes, Mapping, InverseMapping)
    R1 = jR.identifying_r1_mutations_ldoi(GExpanded, output_nodes_ex, mapping, inverse_mapping, input_conditions, IORelation)
#     IIDC = jR.identifying_input_independent_canalizing_mutations(GExpanded, OutputNodes, Mapping, InverseMapping)
#     UN = jR.identifying_input_unreachable_nodes(GExpanded, OutputNodes, Mapping, InverseMapping, InputConditions)
    RN = jR.identifying_rn_mutations(GExpanded, mapping, inverse_mapping, input_nodes_ex, output_nodes_ex, input_conditions,
                                     IORelation,
                                     R0['ineffective_mutations'], R1['r1_mutations'])
    
    NodeList = set(read_nodes_dict.values())
    NodeList.difference_update(input_nodes)
    NodeList.difference_update(output_nodes)
    C0, C1, C2, C3 = [], [], [], []
    for NODE in NodeList:
        negNODE = '~' + NODE
        if R0['ineffective'][NODE] and R0['ineffective'][negNODE]:
#             nodeClass = 'C0'
            C0.append(NODE)
        elif NODE in RN['rn_mutations']:
            if RN['rn_mutations'][NODE] == 'R1':
#                 nodeClass = 'C1'
                C1.append(NODE)
#                 canalizing = IIDC['iid_canalizing'][NODE]
#                 unreachable = UN['input_unreachable'][NODE]
            else:
#                 nodeClass = 'C2'
                C2.append(NODE)
#                 canalizing = IIDC['iid_canalizing'][NODE]
#                 unreachable = UN['input_unreachable'][NODE]
        elif negNODE in RN['rn_mutations']:
            if RN['rn_mutations'][negNODE] == 'R1':
#                 nodeClass = 'C1'
                C1.append(NODE)
#                 canalizing = IIDC['iid_canalizing'][NODE]
#                 unreachable = UN['input_unreachable'][NODE]
        else:
#             nodeClass = 'C3'
            C3.append(NODE)
#             canalizing = IIDC['iid_canalizing'][NODE]
#             unreachable = UN['input_unreachable'][NODE]
    

    robustness_pa = rb.robustness_primary_attractor(g_read=g, state_num=2**sampling_num)
    robustness_ip = rb.robustness_initial_perturbation(g_read=g, state_num=2**sampling_num)
    
    robustness = []
    for key, val in robustness_pa.items():
        robustness.append(val['oe'])
        robustness.append(val['ko'])
#     print(np.mean(robustness))
    
    LDOI_union = set()
    for LDOI in LDOIs[0,:]:
        LDOI_union = LDOI_union.union(LDOI)
    LDOI_union = LDOI_union.union([mapping[x] for x in input_conditions.reshape((2*num_inputs,))])
    GSub = nx.subgraph(GExpanded, LDOI_union)
    io_count = 0
    io_pathway_count = 0
    for i in range(num_input_conditions):
        # G_copied = G_expanded.copy()
        BinaryBit = np.binary_repr(i, width=num_inputs)
        InputCondition = [input_conditions[x, int(BinaryBit[x])] for x in range(num_inputs)]
        # source_nodes = input_condition + constant_nodes
        Sources = set([x for x in InputCondition])
        # for node in source:
        #     G_copied.remove_edges_from(list(G_copied.in_edges(node)))
        #     G_copied.remove_edges_from(list(G_copied.in_edges(BDOId.Negation_in_expanded(node))))

        Targets = set([x for x in IORelation[0, i]])
        if len(Targets) > 0:
            for s in Sources:
                for t in Targets:
                    if nx.has_path(G=g, source=inverse_read_nodes_dict[s.replace('~', '')], target=inverse_read_nodes_dict[t.replace('~', '')]):
                        io_count += 1
                        len_shortest_path = nx.shortest_path_length(G=g, source=inverse_read_nodes_dict[s.replace('~', '')], target=inverse_read_nodes_dict[t.replace('~', '')])
                        for p in nx.all_simple_paths(G=g, source=inverse_read_nodes_dict[s.replace('~', '')], target=inverse_read_nodes_dict[t.replace('~', '')], cutoff=2*len_shortest_path):
                            io_pathway_count += 1

    if io_count == 0:
        redundancy = -1
    else:
        redundancy = io_pathway_count/float(io_count)
    
    results = results.append({'network_idx': network_idx, 'size_of_network': len(g), 'num_link': len(g.edges), 'C0': len(C0), 
                            'C1': len(C1), 'C2': len(C2), 'C3': len(C3), 'robustness_pa': np.mean(robustness), 'robustness_ip': robustness_ip, 
                            'redundancy': redundancy}, ignore_index=True)
    # , 'deterministic_io','nondeterministic_io' 추가 예정
    

Process ForkProcess-96:
Process ForkProcess-97:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/jijoo/miniconda3/envs/theor/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()


KeyboardInterrupt: 

  File "/home/jijoo/miniconda3/envs/theor/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/jijoo/miniconda3/envs/theor/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jijoo/miniconda3/envs/theor/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jijoo/miniconda3/envs/theor/lib/python3.8/concurrent/futures/process.py", line 233, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/jijoo/miniconda3/envs/theor/lib/python3.8/concurrent/futures/process.py", line 233, in _process_worker
    call_item = call_queue.get(block=True)
  File "/home/jijoo/miniconda3/envs/theor/lib/python3.8/multiprocessing/queues.py", line 96, in get
    with self._rlock:
  File "/home/jijoo/miniconda3/envs/theor/lib/python3.8/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__

In [142]:
results

,network_idx,size_of_network,num_link,robustness_pa,robustness_ip,redundancy,C0,C1,C2,C3
0,0.0,25.0,67.0,0.375000,0.679492,-1.00,17.0,5.0,2.0,1.0
1,1.0,25.0,63.0,0.446667,0.831680,10.75,4.0,8.0,0.0,13.0
2,2.0,25.0,58.0,0.660000,0.660586,-1.00,25.0,0.0,0.0,0.0
3,3.0,25.0,69.0,0.420000,0.812578,5.00,13.0,6.0,0.0,6.0
4,4.0,25.0,60.0,0.463333,0.879023,-1.00,23.0,1.0,0.0,1.0
5,5.0,25.0,57.0,0.381000,0.866211,-1.00,25.0,0.0,0.0,0.0
6,6.0,25.0,64.0,0.660000,0.960000,-1.00,19.0,5.0,0.0,1.0
7,7.0,25.0,55.0,0.500000,0.786836,-1.00,20.0,4.0,0.0,1.0
8,8.0,25.0,61.0,0.283529,0.845352,-1.00,20.0,5.0,0.0,0.0
9,9.0,25.0,61.0,0.433333,0.753125,-1.00,24.0,0.0,0.0,1.0


In [143]:
results.to_csv('../data/random_networks_er_kauffman_1.csv')